In [ ]:
import nltk
import numpy as np
from __future__ import division
import pandas as pd
import math
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import wordnet as wn

In [266]:
# path='C:/Users/user/2017_English_final/2017_English_final/GOLD/Subtask_A/'
path='/Users/munde/PycharmProjects/cs474/data'
def extract_txt(files):
    raw=[]
    for f in files:
        raw=raw+open(path+'%s'%f,'r').readlines()
    lines=[a.strip() for a in raw]
    return lines

def create_table(filenames):
    #files have duplicate rows
    txt=extract_txt(filenames)#['twitter-2013dev-A.txt','twitter-2013train-A.txt','twitter-2015train-A.txt'])
    table=pd.DataFrame()
    ids=[]
    tweets=[]
    polarity=[]
    for t in txt:
        t=t.split('\t')
        #tweet id
        t_id=t[0]
        #tweet polarity
        t_pol=t[1]
        #tweet text
        t_text=t[2]
        ids.append(t_id)
        tweets.append(t_text)
        polarity.append(t_pol)
    table['ID']=ids
    table['TWEET']=tweets
    table['POLARITY']=polarity
    #drop duplicates
    table.drop_duplicates('ID', inplace=True)
    return table
#s_class=sentiment class: 'neutral','positive','negative'
def preprocess(table,s_class):
    #get subset of table where polarity == to s_class
    table=table.loc[table['POLARITY']==s_class]
    tokens=[]
    for index,values in table.iterrows():
        tokens=tokens+extract_tokens(values['TWEET'])
    return tokens
def extract_tokens(txt):
    txt=txt.lower()
    st_words=nltk.corpus.stopwords.words('english')
    
    tweet_tok=nltk.TweetTokenizer()
    tokens=tweet_tok.tokenize(txt)
    #tokens= [x for x in tokens if not re.match('[' + string.punctuation + ']+', x)]
    #remove stop words
    tokens=[x for x in tokens if x not in st_words]
    #attach pos tag extract only nouns, verbs, adjectves
    tags= ['JJ','JJR','JJS','NN','NNS','NNP','NNPS','RB','RBR','RBS','VB','VBD','VBG','VBN','VBP','VBZ']
    tokens=[(token,tag) for token, tag in nltk.pos_tag(tokens) if tag in tags]
    return tokens


In [232]:
t=create_table()
positive_tokens=preprocess(t,'positive')
negative_tokens=preprocess(t,'negative')
neutral_tokens=preprocess(t,'neutral')

In [233]:
#filter tokens: leave only those that appear in one group..later use pmi or idf to filter 
def filter_tokens(tokens, tokens_to_remove):
    uniq_t=set(tokens)-set(tokens_to_remove)
    tokens=[t for t in tokens if t in uniq_t]
    return tokens
positive_tokens=filter_tokens(positive_tokens,negative_tokens+neutral_tokens)
negative_tokens=filter_tokens(negative_tokens,positive_tokens+neutral_tokens)
neutral_tokens=filter_tokens(neutral_tokens,negative_tokens+positive_tokens)

In [245]:
def add_synonyms(tokens):
    synonyms=[]
    for token, tag in tokens:
        synonyms=synonyms+get_syns(token,tag)
    #append synonyms
    tokens=set(tokens+synonyms)
    return tokens
def add_antonyms(tokens, antonym_tokens):
    antonyms=[]
    for token, tag in tokens:
        antonyms=antonyms+get_ant(token,tag)
    #append antonyms
    antonym_tokens=set(list(antonym_tokens)+antonyms)
    return antonym_tokens
def get_ant(token,tag):
    synsets=[]
    if tag.startswith('N'):
        synsets=wn.synsets(token, 'n')
    elif tag.startswith('J'):
        synsets=wn.synsets(token, 'a')
    elif tag.startswith('R'):
        synsets=wn.synsets(token, 'r')
    else:
        synsets=wn.synsets(token, 'v')
    antonyms=[]
    for s in synsets:
        lemmas=s.lemmas()
        for l in lemmas:
            antonyms= antonyms+[(w.name(),tag) for w in l.antonyms()]
    return antonyms

def get_syns(token,tag):
    synsets=[]
    if tag.startswith('N'):
        synsets=wn.synsets(token, 'n')
    elif tag.startswith('J'):
        synsets=wn.synsets(token, 'a')
    elif tag.startswith('R'):
        synsets=wn.synsets(token, 'r')
    else:
        synsets=wn.synsets(token, 'v')
    synonyms=[]
    for s in synsets:
        synonyms= synonyms+[(w.name(),tag) for w in s.lemmas()]
    return synonyms

In [235]:
#adding synonyms
positive_tokens=add_synonyms(positive_tokens)
negative_tokens=add_synonyms(negative_tokens)
neutral_tokens=add_synonyms(neutral_tokens)

In [246]:
#adding antonyms
negative_tokens=add_antonyms(positive_tokens,negative_tokens)
positive_tokens=add_antonyms(negative_tokens,positive_tokens)

In [255]:
type(neutral_tokens)

set

In [258]:
#filtering
positive_tokens=filter_tokens(positive_tokens,set(negative_tokens)|set(neutral_tokens))
negative_tokens=filter_tokens(negative_tokens,set(positive_tokens)|neutral_tokens)
neutral_tokens=filter_tokens(neutral_tokens,set(negative_tokens)|set(positive_tokens))

In [263]:
#number of tokens in each class
print len(positive_tokens), len(negative_tokens) , len(neutral_tokens)  

14782 9825 54240


In [269]:
#remove tags
positive_tokens=[token for token, tag in positive_tokens]
negative_tokens=[token for token, tag in negative_tokens]
neutral_tokens=[token for token, tag in neutral_tokens]
def calculate_polarity(tweet_tokens):
    sentiment=0
    for t in tweet_tokens:
        if t in positive_tokens:
            sentiment=sentiment+1
        elif t in negative_tokens:
            sentiment=sentiment-1
    if sentiment>0:
        return 'positive'
    elif sentiment<0:
        return 'negative'
    else:
        return 'neutral'

In [271]:
#testing 
test_t=create_table(['twitter-2013test-A.txt','twitter-2013test-A.txt','twitter-2013test-A.txt'])
sent_class=[]
for index,values in test_t.iterrows():
    tokens=[t for t,tag in extract_tokens(values['TWEET'])]
    sentiment=calculate_polarity(tokens)
    sent_class.append(sentiment)
test_t['Class']=sent_class

In [273]:
classified_correctly=0
for index,values in test_t.iterrows():
    
    if values['POLARITY']==values['Class']:
        classified_correctly+=1
#accuracy
print classified_correctly/len(test_t)

0.422849083216
